In [1]:
import os
import sys
import time
import datetime
import socket
import yaml
import shutil

import pandas as pd
import numpy as np

from sklearn.externals import joblib 

from sklearn.metrics import accuracy_score, log_loss, f1_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
folderFeature = 'feature'
feaPaths = ['../xprmt/'+folderFeature+'/extract-haralick-DataPH2_lesion-mlbox-20181117-171415/DataPH2_lesion',
           '../xprmt/'+folderFeature+'/extract-slbp-DataPH2_lesion-mlbox-20181117-170250/DataPH2_lesion',
           '../xprmt/'+folderFeature+'/extract-wtio-DataPH2_lesion-mlbox-20181118-033549/DataPH2_lesion',
          ]

featureList = []
labelList = []
for feaPath in feaPaths:
    print(feaPath)
    xListL = []
    yListL = []
    yLi = os.listdir(feaPath)
    sumdat = 0
    for i in yLi:
        file = os.listdir(feaPath+ "/" + i)
        cnt = 0
        for j in file:
            if j.endswith('.pkl'):
                #print(j)
                x = joblib.load(feaPath + "/" + i + "/" +j)
                xListL.append(x)
                yListL.append(i)
                cnt += 1
        print(str(i) + " " + str(cnt))
        sumdat += cnt
    
    print("semuanya "+ str(sumdat))
    featureList.append(xListL)
    labelList.append(yListL)

xList = np.concatenate((featureList[0], featureList[1]), axis=1)
for idx in range(len(featureList)-2):
    xList = np.concatenate((xList, featureList[idx+2]), axis=1)

xList1 = xList
yList1 = labelList[0]

../xprmt/feature/extract-haralick-DataPH2_lesion-mlbox-20181117-171415/DataPH2_lesion
Melanoma 40
Atypical Nevus 80
Common Nevus 80
semuanya 200
../xprmt/feature/extract-slbp-DataPH2_lesion-mlbox-20181117-170250/DataPH2_lesion
Melanoma 40
Atypical Nevus 80
Common Nevus 80
semuanya 200
../xprmt/feature/extract-wtio-DataPH2_lesion-mlbox-20181117-165159/DataPH2_lesion
Melanoma 40
Atypical Nevus 80
Common Nevus 80
semuanya 200


In [3]:
folderFeature = 'feature'
feaPaths = ['../xprmt/'+folderFeature+'/extract-haralick-DataPH2_lesion_hairremove-mlbox-20181117-174657/DataPH2_lesion_hairremove',
           '../xprmt/'+folderFeature+'/extract-slbp-DataPH2_lesion_hairremove-mlbox-20181117-173341/DataPH2_lesion_hairremove',
           '../xprmt/'+folderFeature+'/extract-wtio-DataPH2_lesion_hairremove-mlbox-20181118-035201/DataPH2_lesion_hairremove',
          ]

featureList = []
labelList = []
for feaPath in feaPaths:
    print(feaPath)
    xListL = []
    yListL = []
    yLi = os.listdir(feaPath)
    sumdat = 0
    for i in yLi:
        file = os.listdir(feaPath+ "/" + i)
        cnt = 0
        for j in file:
            if j.endswith('.pkl'):
                #print(j)
                x = joblib.load(feaPath + "/" + i + "/" +j)
                xListL.append(x)
                yListL.append(i)
                cnt += 1
        print(str(i) + " " + str(cnt))
        sumdat += cnt
    
    print("semuanya "+ str(sumdat))
    featureList.append(xListL)
    labelList.append(yListL)

xList = np.concatenate((featureList[0], featureList[1]), axis=1)
for idx in range(len(featureList)-2):
    xList = np.concatenate((xList, featureList[idx+2]), axis=1)

xList2 = xList
yList2 = labelList[0]

../xprmt/feature/extract-haralick-DataPH2_lesion_hairremove-mlbox-20181117-174657/DataPH2_lesion_hairremove
Melanoma 40
Atypical Nevus 80
Common Nevus 80
semuanya 200
../xprmt/feature/extract-slbp-DataPH2_lesion_hairremove-mlbox-20181117-173341/DataPH2_lesion_hairremove
Melanoma 40
Atypical Nevus 80
Common Nevus 80
semuanya 200
../xprmt/feature/extract-wtio-DataPH2_lesion_hairremove-mlbox-20181117-172037/DataPH2_lesion_hairremove
Melanoma 40
Atypical Nevus 80
Common Nevus 80
semuanya 200


In [4]:
xList = np.concatenate((xList1, xList2), axis=0)
yList1.extend(yList2)
yList = yList1
print(len(yList))

400


In [5]:
xList = np.array(xList)
yList = np.array(yList)
print(xList.shape)
print(yList.shape)

(400, 2079)
(400,)


In [6]:
from sklearn.preprocessing import normalize
xList = normalize(xList, norm='l2', axis=0)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xList, yList, random_state = 42, test_size = 0.2)

In [8]:
print(X_train.shape)

(320, 2079)


In [9]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import sensitivity_specificity_support
clf = RandomForestClassifier(random_state=6)
clf.fit(X_train, y_train)
prediksi = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, prediksi))
print(classification_report(y_test, prediksi))
print(sensitivity_specificity_support(y_test, prediksi, average='macro'))

0.9
                precision    recall  f1-score   support

Atypical Nevus       0.93      0.81      0.87        32
  Common Nevus       0.88      0.94      0.91        31
      Melanoma       0.89      1.00      0.94        17

     micro avg       0.90      0.90      0.90        80
     macro avg       0.90      0.92      0.91        80
  weighted avg       0.90      0.90      0.90        80

(0.915994623655914, 0.948318216175359, None)


In [10]:
clf1 = RandomForestClassifier(random_state=1)
clf2 = GradientBoostingClassifier()
clf3 = LinearDiscriminantAnalysis()
eclf1 = VotingClassifier(estimators=[('rf', clf1), ('gb', clf2), ('lda', clf3)], voting='hard')
score = cross_val_score(clf, xList, yList, cv=10)
print(score)
print(score.mean())

[0.825 0.85  0.975 0.9   0.9   0.875 0.8   0.9   0.95  0.9  ]
0.8875


In [11]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
score = cross_val_score(clf, xList, yList, cv=10)
print(score)
print(score.mean())

[0.925 0.775 0.9   0.975 0.825 0.85  0.75  0.925 0.825 0.875]
0.8625


In [12]:

classifiers = [
    KNeighborsClassifier(3),
    #SVC(kernel="rbf", C=0.025, probability=True),
    #NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(random_state=1),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
    ######################### perform 10 fold validation ######################
    kf = StratifiedKFold(n_splits=10)
    scorelist = []
    sensitivity = []
    specificity = []
    f1 = []
    for train_index, test_index in kf.split(xList, yList):
        clf.fit(xList[train_index], yList[train_index])
        p = clf.predict(xList[test_index])
        accval = accuracy_score(yList[test_index], p)
        scorelist.append(accval)
        sss = sensitivity_specificity_support(yList[test_index], p, average='macro')
        sensitivity.append(sss[0])
        specificity.append(sss[1])
        f1.append(f1_score(yList[test_index], p, average='macro') )

        
    print("MeanCVScore: {}".format(sum(scorelist)/len(scorelist)))
    print("10FoldCVScore: {}".format(scorelist))
    print("sensitivity: {}".format(sum(sensitivity)/len(sensitivity)))
    print("specificity: {}".format(sum(specificity)/len(specificity)))
    print("f1-score: {}".format(sum(f1)/len(f1)))

    #############################################################################
    
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 66.2500%
Log Loss: 0.4656073226554728
MeanCVScore: 0.6475
10FoldCVScore: [0.65, 0.65, 0.675, 0.625, 0.625, 0.65, 0.625, 0.675, 0.625, 0.675]
sensitivity: 0.6583333333333333
specificity: 0.809375
f1-score: 0.6650308204848552
DecisionTreeClassifier
****Results****
Accuracy: 77.5000%
Log Loss: 7.771224688854905
MeanCVScore: 0.8625
10FoldCVScore: [0.85, 0.875, 0.925, 0.85, 0.825, 0.875, 0.85, 0.9, 0.775, 0.9]
sensitivity: 0.8854166666666666
specificity: 0.9249999999999998
f1-score: 0.8800235946053739
RandomForestClassifier
****Results****
Accuracy: 80.0000%
Log Loss: 0.4947835339145227
MeanCVScore: 0.9
10FoldCVScore: [0.95, 0.9, 0.925, 0.925, 0.925, 0.875, 0.875, 0.85, 0.875, 0.9]
sensitivity: 0.9124999999999999
specificity: 0.9447916666666666
f1-score: 0.9131802275101201
AdaBoostClassifier
****Results****
Accuracy: 62.5000%
Log Loss: 0.6371681117322445
MeanCVScore: 0.6475
10FoldCVScore: [0.675, 0.65, 0.65, 0.625, 0.725, 0.625, 0.475, 0.65, 0.

In [13]:
# feaPath = '../xprmt/feature1/extract-slbp-DataPH2-mlbox-20181116-021021/DataPH2'

# xListL = []
# yListL = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListL.append(x)
#             yListL.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))
# feaPath = '../xprmt/feature1/extract-haralick-DataPH2-mlbox-20181116-021704/DataPH2'

# xListH = []
# yListH = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListH.append(x)
#             yListH.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))
# feaPath = '../xprmt/feature1/extract-wtio-DataPH2-mlbox-20181116-021739/DataPH2'

# xListW = []
# yListW = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListW.append(x)
#             yListW.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))

# xListL = np.array(xListL)
# xListH = np.array(xListH)
# xListW = np.array(xListW)
# xList2 = np.concatenate((xListL, xListH, xListW), axis=1)
# #xList = np.concatenate((xListL, xListW), axis=1)
# print(yListL == yListH)
# print(yListH == yListW)
# yList2 = yListL